## Scraping module
Scraping of straits times news article using selenium that are returned from the search function (keyword - 'food supply')

### Imports

In [31]:
#!pip install nltk
#import nltk
#nltk.download('punkt')

In [32]:
#!pip install newspaper3k

In [33]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

from newspaper import Article
import pandas as pd
import datetime

### Newspaper Article Summariser

In [34]:
def getSummary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    full_text = article.text
    location = full_text[0:full_text.find("-")]
    return article.summary, article.keywords, article.publish_date, location

### StraitsTimes Scraper

In [35]:
# change to ur own directory!
driver = webdriver.Chrome("C:/Users/jonat/chromedriver/chromedriver.exe") 

def getNews():
    output = []
    
    # get news title in a page - need a for loop to get all the pages!
    all_news_per_page=WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"queryly_item_title")))
    
    # get news url (i.e. news source) & news summary & keywords using the summariser module
    for i in range(len(all_news_per_page)):
        news_details = []
        xpath = f'//*[@id="resultdata"]/div[{i+2}]/a'        
        url = all_news_per_page[i].find_element_by_xpath(xpath).get_attribute('href')
        try: 
            summary = getSummary(url)
            try:
                date = summary[2].strftime('%d/%m/%Y')[3:]
            except:
                date = "NA"
            news_details.append(date)
            news_details.append(summary[3])
            news_details.append(all_news_per_page[i].text)
            news_details.append(url)
            news_details.append(summary[0])
            news_details.append(", ".join(summary[1]))
            output.append(news_details)
        except: 
            continue
        
    return output

C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [36]:
def getStraitsTimesNews(searchTerm):
    base_url = "https://www.straitstimes.com/search?searchkey=" + searchTerm.replace(" ", "%20")
    driver.get(base_url)

    # close ad (if any)
    try:
        iframe=driver.find_elements_by_tag_name("iframe")[3]
        driver.switch_to.frame(iframe)
        driver.find_elements_by_tag_name("iframe")
        ad = driver.switch_to.frame(0)
        driver.find_element_by_id('pclose-btn').click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()
    
    # filter the search result by date by xpath accordingly
    # within 24 hours, within one week, within 30 days, within a year, within three years
    filter_by_date = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,"//*[contains(text(), 'within three years')]")))[0].click()

    # get news on page
    output = getNews()
    
    # check for next page
    nextpage = True
    while (nextpage):  
        try:
            nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,'#resultdata > a > h2')))[0]
            if (nextpage.text == 'Next Page'):
                nextpage.click()
#                 print('clicked next')
                output.extend(getNews())
            else: 
                nextpage = False
        except:
#             print("no next page!")
            nextpage = False
    
    
    # convert to df and export as csv
    df = pd.DataFrame(output, columns=['date', 'location','news title', 'news source(url)', 'content summary', 'keywords'])
    df['class_name'] = "trade dispute"
    df.to_csv(f"csv_data/{searchTerm}_straitstimes.csv")
    return df

getStraitsTimesNews("tradedispute")

C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  import sys
C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  if __name__ == '__main__':
C:\Users\jonat\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


,date,location,news title,news source(url),content summary,keywords,class_name
0,02/2022,MUNICH (NYTIMES),Would it be a massive strike or minor incursio...,https://www.straitstimes.com/world/europe/woul...,"Hours before, Mr Biden had informed them that ...","russia, incursion, massive, russian, biden, pr...",trade dispute
1,02/2022,PARIS (REUTERS),Major strike at Paris metro disrupts daily com...,https://www.straitstimes.com/world/europe/majo...,PARIS (REUTERS) - A major strike paralysed mos...,"millions, workers, unions, reform, ratp, paris...",trade dispute
2,02/2022,"DONETSK, UKRAINE/MOSCOW (REUTERS) – Russia’s s...","Putin oversees nuclear drills, US says Russian...",https://www.straitstimes.com/world/europe/puti...,"DONETSK, UKRAINE/MOSCOW (REUTERS) – Russia’s s...","austin, zelenskiy, russian, oversees, presiden...",trade dispute
3,02/2022,DAMASCUS (AFP),Israel strikes town south of Damascus: Syrian ...,https://www.straitstimes.com/world/middle-east...,DAMASCUS (AFP) - Israeli shelling struck a tow...,"monitor, shelling, targeted, media, syrian, st...",trade dispute
4,02/2022,LONDON (REUTERS),Horse racing: Olympic champion Todd suspended ...,https://www.straitstimes.com/sport/horse-racin...,LONDON (REUTERS) - Twice Olympic equestrian ch...,"horse, video, trains, london, racing, suspensi...",trade dispute
...,...,...,...,...,...,...,...
1810,11/2021,WHO classifies new Covid,While You Were Sleeping: 5 stories you might h...,https://www.straitstimes.com/world/while-you-w...,WHO classifies new Covid-19 strain Omicron as ...,"sleeping, variant, asia, nov, cent, countries,...",trade dispute
1811,05/2019,A spokesman for the US Trade Representative's ...,Trump tariff threat leaves US-China talks in l...,https://www.straitstimes.com/business/companie...,Tensions between the two countries had eased a...,"threat, talks, markets, tariffs, limbo, trump,...",trade dispute
1812,02/2022,MUSCAT (AFP),Oman slaps hefty fines on men wearing rogue ro...,https://www.straitstimes.com/world/middle-east...,MUSCAT (AFP) - A wave of foreign imitations an...,"robes, white, usually, slit, protect, fines, n...",trade dispute
1813,08/2020,"GAZA CITY, PALESTINIAN TERRITORIES (AFP)",Israeli planes bomb Gaza as Egyptians seek to ...,https://www.straitstimes.com/world/middle-east...,"GAZA CITY, PALESTINIAN TERRITORIES (AFP) - Isr...","strip, seek, egyptians, talks, response, calm,...",trade dispute
